# Setup Dependencies

In [1]:
from transformers import pipeline
import kagglehub
import os
import pandas as pd
from utils import get_files_in_dir
from pandasql import sqldf

# Get Model and Data

In [4]:
model_name = "openai/clip-vit-large-patch14-336"
classifier = pipeline("zero-shot-image-classification", model = model_name, use_fast=True)

path = kagglehub.dataset_download("gpreda/happy-mammals-with-128x128-image-size")

print("Path to dataset files:", path)

image_paths =  get_files_in_dir(f"{path}/train_images_128")
labels_path="train.csv"

labels = pd.read_csv(labels_path, index_col="image")

# Collect all possible labels and setup results table

In [8]:
possible_labels=labels["species"].unique()
print(f"Number of categories:{len(possible_labels)}")
print(f"Categorizing images into the following species: {", ".join(possible_labels)}")

results=pd.DataFrame(columns=["image_name","label", "prediction","is_prediction_correct"])
results["is_prediction_correct"].astype(bool)

# Generate all predictions

In [6]:
image_names = [os.path.basename(image_path) for image_path in image_paths]

true_labels = labels.loc[image_names, "species"].tolist()

batch_size = 8
max_num_batches=100

for i in range(0, len(image_paths), batch_size):
    batch_num=i // batch_size + 1
    
    if batch_num >= max_num_batches:
        break

    batch_image_paths = image_paths[i:i + batch_size]
    batch_image_names = image_names[i:i + batch_size]
    batch_true_labels = true_labels[i:i + batch_size]

    print(f"Generating scores for batch {batch_num}")
    batch_scores = classifier(batch_image_paths, candidate_labels=possible_labels)

    for image_name, label, scores in zip(batch_image_names, batch_true_labels, batch_scores):
        prediction = scores[0]['label']
        is_prediction_correct = label == prediction

        result=pd.DataFrame([{
            "image_name": image_name,
            "is_prediction_correct": is_prediction_correct,
            "label": label,
            "prediction": prediction
        }])
        results = pd.concat([results, result], ignore_index=True)


    print(f"Processed batch {batch_num}/{(len(image_paths) + batch_size - 1) // batch_size}")

print("Classification complete.")

# Accuracy Analysis

In [7]:
print("")
numCorrect=results["is_prediction_correct"].sum()
accuracy=numCorrect/len(results["is_prediction_correct"])
print(f"num correct: {numCorrect} Accuracy: {accuracy}")


query1="""SELECT label, 
COUNT(CASE WHEN is_prediction_correct = FALSE THEN 1 END) AS num_incorrect, 
COUNT(*) AS total,
ROUND(CAST(COUNT(CASE WHEN is_prediction_correct = TRUE THEN 1 END) AS FLOAT)/COUNT(label),3) AS label_accuracy
FROM results 
GROUP BY label 
ORDER BY label_accuracy DESC"""
sqldf(query1)
